[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_Code/02_Reacher_Env/01_Generate_Trajectories/01_Generating_Reacher_Trajectories.ipynb)

In [ ]:
!pip install gymnasium mujoco stable-baselines3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import the necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from google.colab import data_table
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to save the training logs
log_dir = "/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/00_training_logs/0_log_dir/0_ppo_reacher_initial"            # Update directory location 1
os.makedirs(log_dir, exist_ok=True)

In [ ]:
# Create the environment
env = gym.make("Reacher-v5")
env = Monitor(env, log_dir)

In [ ]:
# Step 1.2: Initial Observation
# Print out a sample observation to see what the agent receives at the start.
obs = env.reset()
print("Initial Observation: ", obs)

Initial Observation:  (array([ 9.98233483e-01,  9.99624240e-01, -5.94130706e-02, -2.74112890e-02,
        1.75126402e-01,  8.64263427e-02,  4.45406412e-03,  9.32875796e-04,
        3.42822237e-02, -1.01910547e-01]), {})


In [ ]:
# Step 1.3: Implement PPO with the Highway environment
# Initialize the PPO algorithm with the chosen environment.
# Create the PPO model with the Highway environment
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_dir)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [ ]:
# Step 1.4: Train the model
# 200,000 timesteps for initial training
model.learn(total_timesteps=200000)

Logging to /content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/00_training_logs/0_log_dir/0_ppo_reacher_initial/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -60.5    |
| time/              |          |
|    fps             | 949      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 50         |
|    ep_rew_mean          | -60.3      |
| time/                   |            |
|    fps                  | 677        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00542826 |
|    clip_fraction        | 0.0362     |
|    clip_range           | 0.2        |
|  

In [ ]:
log_path = os.path.join(log_dir, "monitor.csv")
df = pd.read_csv(log_path, skiprows=1)
df.rename(columns={"index": "episode", "r": "reward", "l": "length", "t": "time_step"}, inplace=True)

In [ ]:
# Step 1.5: Save the trained model to Google Drive
model.save('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/01_trained_models/0_ppo_reacher_intial_training')           # Update directory location 2

In [ ]:
# Step 1.6: Load the saved PPO model from Google Drive
model = PPO.load('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/01_trained_models/0_ppo_reacher_intial_training')     # Update directory location 3

# TRAJECTORY GENERATION AND COLLECTION

In [ ]:
def generate_reacher_trajectories(env, model, num_episodes):
    """
    Generates trajectories for the Reacher-v5 environment.

    Parameters:
        env (gym.Env): The wrapped Gymnasium environment.
        model (stable_baselines3.PPO): The trained PPO model.
        num_episodes (int): Number of episodes to run.

    Returns:
        pd.DataFrame: A DataFrame containing trajectory data.
    """
    trajectory_data = []

    for episode in range(num_episodes):
        state, _ = env.reset()
        timestep = 0
        done = False

        while not done:
            # Get action from the trained PPO model
            action, _ = model.predict(state)

            next_state, reward, terminated, truncated, info = env.step(action)

            # Store the transition
            trajectory_data.append({
                "Episode": episode + 1,
                "Timestep": timestep,
                "State": state,
                "Action": action,
                "Reward": reward,
                "Next State": next_state
            })

            # Update state and timestep
            state = next_state
            timestep += 1
            done = terminated or truncated

    return pd.DataFrame(trajectory_data)

In [ ]:
trajectory_df = generate_reacher_trajectories(env, model, num_episodes=100)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Check the data type of each column
print(type(trajectory_df['Episode'][0]))
print(type(trajectory_df['Timestep'][0]))
print(type(trajectory_df['State'][0]))
print(type(trajectory_df['Action'][0]))
print(type(trajectory_df['Reward'][0]))
print(type(trajectory_df['Next State'][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>


In [ ]:
# Define the path to save the DataFrame (adjust the path as necessary)
trajectory_df_path = '/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/02_trajectories/0_initial_training/0_initial_trajectory_reacher_df.csv'         # Update directory location 4
# Save the DataFrame to Google Drive as a CSV file
trajectory_df.to_csv(trajectory_df_path, index=False)

In [ ]:
# Save the processed dataframe as a pickle file
trajectory_df.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/02_trajectories/0_initial_training/0_initial_trajectory_reacher_df.pkl')      # Update directory location 5